In [1]:
import pandas as pd
import numpy as np


file = "cleaned_v2.csv"

df = pd.read_csv(file)


# Step 1: Extract individual options from the 'options' column
def extract_options(option_string, option_type):
    if isinstance(option_string, str):  # Check if the option_string is a valid string
        start = option_string.find(option_type)
        if start != -1:
            start_index = start + len(option_type) + 2  # Adjust to start after ": "
            end_index = option_string.find("\n", start_index)  # Find the end of this section
            if end_index == -1:
                end_index = len(option_string)  # Handle the case if it's the last section
            options = option_string[start_index:end_index].strip()
            return options
    return None


# Apply the function to extract options and create new columns
df["audio_technology"] = df["options"].apply(lambda x: extract_options(x, "audio si tehnologie"))
df["comfort_equipment"] = df["options"].apply(lambda x: extract_options(x, "confort si echipamente optionale"))
df["electronics_assistance"] = df["options"].apply(lambda x: extract_options(x, "electronice si sisteme de asistenta"))
df["performance"] = df["options"].apply(lambda x: extract_options(x, "performanta"))
df["safety"] = df["options"].apply(lambda x: extract_options(x, "siguranta"))

# Step 2: Drop the original 'options' column
df = df.drop(columns=["options"])

# Step 3: Handle missing values in the new option columns by filling with 'None' (or any other placeholder)
df["audio_technology"] = df["audio_technology"].fillna("None")
df["comfort_equipment"] = df["comfort_equipment"].fillna("None")
df["electronics_assistance"] = df["electronics_assistance"].fillna("None")
df["performance"] = df["performance"].fillna("None")
df["safety"] = df["safety"].fillna("None")


df.head()

manufacturer     model    year      km    fuel  power  engine_capacity   
0        renault  talisman  2016.0  158271  diesel    160             1598  \
1  mercedes-benz         e  2006.0   73867  diesel    190             2987   
2          skoda   octavia  2019.0  178856  diesel    183             1968   
3        renault      clio  2016.0  228000  diesel     90             1461   
4           ford      kuga  2012.0  216000  diesel    163             1997   

    transmission  automatic chassis country of origin   
0           fata       True   sedan            franta  \
1          spate       True   sedan            italia   
2  4x4 (automat)       True   combi           romania   
3           fata      False   combi               NaN   
4  4x4 (automat)       True     suv            olanda   

                                         description  price currency  id   
0  renault talisman 1.6 dci energy intens  - 160c...  17490      eur   8  \
1  - mercedes-benz e-class e280 cdi - dric/limuzi...  23969      eur  10   
2  garantie 24 luni in limita a 30.000 km\n\nposi...  16789      eur  11   
3  renault clio iv limited\nmotor: 1.5 dci , 90 c...   7890      eur  12   
4  ford kuga \nfab- 2012\nmotor diesel - 163cp\na...   8700      eur  13   

                                    audio_technology   
0  apple carplay,android auto,bluetooth,radio,sis...  \
1                  bluetooth,radio,sistem hands-free   
2  apple carplay,android auto,bluetooth,radio,sis...   
3                                               None   
4  bluetooth,radio,port usb,sistem navigatie,sist...   

                                   comfort_equipment   
0  carlig remorcare,climatronic,jaluzele manuale ...  \
1  climatronic,tapiterie stofa,scaun sofer ajusta...   
2  climatronic 2 zone,tapiterie piele,incalzire s...   
3     volan reglabil electric,geamuri electrice fata   
4  carlig remorcare,climatronic,plafon panoramic,...   

                              electronics_assistance   
0  pilot automat,senzori parcare fata,senzori par...  \
1  pilot automat,oglinzi exterioare incalzite,lim...   
2  pilot automat,senzori parcare fata,senzori par...   
3  pilot automat adaptiv (distronic),faruri led,o...   
4  pilot automat,faruri xenon,senzori parcare fat...   

                        performance   
0      jante aliaj 19,anvelope vara  \
1  jante aliaj 16,suspensie confort   
2                    jante aliaj 17   
3                        jante otel   
4                              None   

                                              safety  
0  abs,esp,franare asistata,airbag sofer,airbag s...  
1  abs,esp,franare asistata,airbag sofer,airbag s...  
2  abs,esp,franare asistata,airbag sofer,airbag s...  
3  abs,esp,airbag sofer,airbag scaun pasager,airb...  
4  abs,esp,ebd,franare asistata,sistem avertizare...

In [2]:
# Define categorical and numerical columns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Drop country of origin, description and currency columns
df = df.drop(columns=["description"])
df = df.drop(columns=["country of origin"])
df = df.drop(columns=["currency"])

# Define categorical and numerical columns
categorical_columns = [
    "fuel",
    "transmission",
    "chassis",
    "manufacturer",
    "model",
    "audio_technology",
    "comfort_equipment",
    "electronics_assistance",
    "performance",
    "safety",
    "automatic",
]
numerical_columns = ["power", "engine_capacity", "km", "year"]

# Encoding categorical variables
label_encoder = LabelEncoder()
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Feature scaling
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

df.head()

manufacturer  model      year        km  fuel     power  engine_capacity   
0            34    393  0.173785 -0.119980     0  0.110123        -0.498617  \
1            26    151 -2.271356 -1.233023     0  0.660159         2.348137   
2            36    283  0.907327  0.151476     0  0.531817         0.259698   
3            34    115  0.173785  0.799542     0 -1.173294        -0.779398   
4            14    239 -0.804272  0.641297     0  0.165126         0.319133   

   transmission  automatic  chassis  price  id  audio_technology   
0             2          1        6  17490   8               256  \
1             3          1        6  23969  10               560   
2             0          1        1  16789  11               230   
3             2          0        1   7890  12                 0   
4             0          1        7   8700  13               532   

   comfort_equipment  electronics_assistance  performance  safety  
0               3058                    8905          455    3952  
1              11343                    8651          115    3954  
2               5287                    9833          125    3919  
3              12278                    1929          819     391  
4               3070                    7726            0    2851

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

X = df.drop(columns=["price"])
y = df["price"]

# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train, 30% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 15% val, 15% test

# Define the parameter grid for hyperparameter tuning
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of trees in the model
    "learning_rate": [0.01, 0.1, 0.2],  # Step size shrinkage
    "max_depth": [3, 5, 7],  # Maximum depth of trees
    "subsample": [0.8, 1.0],  # Subsample ratio of the training data
    "colsample_bytree": [0.8, 1.0],  # Subsample ratio of columns during tree construction
    "reg_alpha": [0, 0.1, 0.5],  # L1 regularization term
    "reg_lambda": [1, 2, 5],  # L2 regularization term
}

# Create the XGBRegressor model
xgb_model = XGBRegressor(random_state=42)

# Perform the grid search

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Use the best model to make predictions

best_model = grid_search.best_estimator_

# Make predictions on the validation set
y_pred_val = best_model.predict(X_val)

# Make predictions on the test set
y_pred_test = best_model.predict(X_test)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=1

In [4]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Calculate R^2 score
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, y_pred_test)

# Calculate MAE
mae_val = mean_absolute_error(y_val, y_pred_val)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Calculate MSE
mse_val = mean_squared_error(y_val, y_pred_val)
mse_test = mean_squared_error(y_test, y_pred_test)

print("Validation Set:")
print("R^2 Score:", r2_val)
print("Mean Absolute Error:", mae_val)
print("Mean Squared Error:", mse_val)

print("\nTest Set:")
print("R^2 Score:", r2_test)
print("Mean Absolute Error:", mae_test)
print("Mean Squared Error:", mse_test)

Validation Set:
R^2 Score: 0.9325992057533887
Mean Absolute Error: 1583.6591288810716
Mean Squared Error: 5223175.2254444845

Test Set:
R^2 Score: 0.9250790042220822
Mean Absolute Error: 1669.0536012719763
Mean Squared Error: 6001517.2355323285
